In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

In [2]:
df_ASPM = pd.concat(
    [pd.read_csv(ZipFile('ASPM.zip').open(i)) for i in ZipFile('ASPM.zip').namelist()],
    ignore_index=True
)
df_ASPM['Facility'] = df_ASPM['Facility'].str.strip()

df_ASPM.head()

,Hour,GMTHour,Date,Facility,ScheduledDepartures,ScheduledArrivals,DeparturesFor MetricComputation,ArrivalsFor MetricComputation,% On-TimeGateDepartures,% On-TimeAirportDepartures,% On-TimeGateArrivals,AverageGateDepartureDelay,AverageTaxiOutTime,AverageTaxiOutDelay,AverageAirportDepartureDelay,AverageAirborneDelay,AverageTaxiInDelay,AverageBlockDelay,AverageGateArrivalDelay
0,0,4,03/09/2015,ATL,0,3,0,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0
1,0,4,03/09/2015,BOS,1,9,1,1,100.0,100.0,100.0,0.0,9.0,0.0,0.0,1.0,1.7,0.0,10.0
2,0,4,03/09/2015,BWI,0,8,0,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0
3,0,4,03/09/2015,CLT,0,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,03/09/2015,DCA,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
weather_zip = ZipFile('AirportWeather.zip')

df_weather = pd.concat(
    [pd.read_csv(weather_zip.open(i), comment="#", skiprows=[11]) for i in weather_zip.namelist()],
    ignore_index=True
)

df_weather.head()

,Station_ID,Date_Time,air_temp_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,weather_cond_code_set_1,precip_accum_one_hour_set_1,visibility_set_1
0,KATL,2015-01-01T00:52:00Z,5.0,2.57,330.0,NaN,NaN,NaN,10.0
1,KATL,2015-01-01T01:52:00Z,4.0,2.06,310.0,NaN,NaN,NaN,10.0
2,KATL,2015-01-01T02:52:00Z,4.0,0.00,0.0,NaN,NaN,NaN,10.0
3,KATL,2015-01-01T03:52:00Z,3.0,2.57,320.0,NaN,NaN,NaN,10.0
4,KATL,2015-01-01T04:52:00Z,3.0,3.09,320.0,NaN,NaN,NaN,10.0


In [4]:

# for i in range(len(df_ASPM.index)):
#     num_departures = df_ASPM.loc[i, 'DeparturesFor MetricComputation']
#     percent_ontime = df_ASPM.loc[i, '% On-TimeAirportDepartures']
#     ontime_departures = round(num_departures * (percent_ontime/100))
#     for j in range(num_departures):
#         delayed = not (j < ontime_departures)
#         departures.append(df_ASPM.loc[i, ['Hour', 'Date', 'Facility']])

departures = np.empty((0, 4))

for row in df_ASPM[df_ASPM['Facility'] == 'LAX'].itertuples(index=False):
   num_departures = row[6]
   percent_ontime = row[9]
   ontime_departures = round(num_departures * (percent_ontime/100))
   np_row = np.array([row[0], row[2], row[3]])
   ontime = np.full((ontime_departures, 4), np.append(np_row, False))
   delayed = np.full((num_departures - ontime_departures, 4), np.append(np_row, True))
   departures = np.concatenate((departures, ontime, delayed))

KeyboardInterrupt: 

In [13]:
df_weather = df_weather.drop(['wind_gust_set_1'], axis = 1)
df_weather = df_weather.rename(columns = {'air_temp_set_1': 'air_temp','wind_speed_set_1': 'wind_speed','wind_direction_set_1': 'wind_direction',
'weather_cond_code_set_1': 'weather_condition', 'precip_accum_one_hour_set_1': 'precipitation', 'visibility_set_1': 'visibility'})

KeyError: "['wind_gust_set_1'] not found in axis"

In [14]:
df_weather = df_weather.fillna(0)
df_weather['weather_condition'] = df_weather['weather_condition'].astype(int)
df_weather.head()

,Station_ID,Date_Time,air_temp,wind_speed,wind_direction,weather_condition,precipitation,visibility
0,KATL,2015-01-01T00:52:00Z,5.0,2.57,330.0,0,0.0,10.0
1,KATL,2015-01-01T01:52:00Z,4.0,2.06,310.0,0,0.0,10.0
2,KATL,2015-01-01T02:52:00Z,4.0,0.00,0.0,0,0.0,10.0
3,KATL,2015-01-01T03:52:00Z,3.0,2.57,320.0,0,0.0,10.0
4,KATL,2015-01-01T04:52:00Z,3.0,3.09,320.0,0,0.0,10.0


In [17]:
df_weather['weather_condition'].unique()

def find_weather_code(code):
    if code < 80:
        return code, 0, 0
    elif (code >= 80 and code < 6400):
        b = np.floor(code/80)
        c = code - (80*b)
        return 0, b, c
    else:
        a = np.floor(code/6400)
        remainder = code - (6400*a)
        b = np.floor(remainder/80)
        c = remainder - (80*b)
        return a, b, c

print(find_weather_code(200))

(0, 2.0, 40.0)
